In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f'''
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.f_po_pending_fc AS 
SELECT      a.calday,    
            TRY_CAST(a.creation_date as DATE)  creation_date,
            a.store_id,    
            a.article_id product_id,    
            a.unit_code,    
            a.quantity,
            a.quantity * c.numerator/c.denomintR base_qty
FROM        {catalog_name}.udp_wcm_silver_rop.franchise_popending a
LEFT JOIN   {catalog_name}.udp_wcm_gold_vcm_dwh.d_unit_conv c ON a.article_id = c.material AND a.unit_code = c.mat_unit
WHERE       proc_date = (SELECT MAX(proc_date) FROM {catalog_name}.udp_wcm_silver_rop.franchise_popending)
            AND REGEXP_LIKE(store_id, '^F[0-9]') AND store_id LIKE 'F%'
''')